https://github.com/santino-iannone/PytorchDenoisingAE

In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image
import glob
from torch.autograd import Variable

import os
from collections import OrderedDict
from torchvision.models import vgg19

# 코드 실행결과의 동일성을 위해 무작위 시드를 설정합니다
manualSeed = 999
#manualSeed = random.randint(1, 10000) # 만일 새로운 결과를 원한다면 주석을 없애면 됩니다
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Random Seed:  999


In [2]:
batch_size = 1

In [3]:
path = '/home/user303/GAN/COVID-19_Radiography_Dataset/Normal/images'
image_paths = glob.glob(path + '/*.png')

In [4]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(64),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5], std=[0.5])
])

import cv2
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, paths, transform=None):
        self.paths = paths
        self.transform = transform
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, index):
        image_path = self.paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
        
        if self.transform:
            image_tensor = self.transform(image)
            
        return image_tensor
    
train_dataset = ImageDataset(image_paths[:round(len(image_paths)*0.8)], transform)
test_dataset = ImageDataset(image_paths[round(len(image_paths)*0.8):], transform) 

In [5]:
full_dataset = ConcatDataset([train_dataset, test_dataset])
data_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size, shuffle=False)

In [6]:
for i in train_loader:
    print(i)
    break

tensor([[[[0.0078, 0.0000, 0.0000,  ..., 0.1569, 0.2157, 0.3608],
          [0.0039, 0.0000, 0.0000,  ..., 0.0078, 0.0353, 0.1686],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0157, 0.1412],
          ...,
          [0.2941, 0.2039, 0.0471,  ..., 0.4118, 0.5686, 0.6902],
          [0.3098, 0.2353, 0.0627,  ..., 0.4235, 0.5765, 0.6980],
          [0.3451, 0.2824, 0.1216,  ..., 0.4314, 0.5882, 0.7020]]]])


### 모형 정의

In [7]:
np.random.seed(3)
def add_noise(noise_typ,image):
    #Gaussian Noisy
    if noise_typ == "gauss":
        image = np.array(image)
        noisy = image + 0.1 *  np.random.normal(loc=0,scale=1.,size=image.shape)
        noisy = torch.Tensor(noisy)
        return noisy

    #Salt & Pepper Noisy
    elif noise_typ == "s&p":
        bt,ch,row,col= image.shape
        s_vs_p = 0.5
        amount = 0.05
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * 4096 * s_vs_p)
        print(num_salt)
        coords = [np.random.randint(0,i,3) for i in image.shape[1:]]
        print(coords)
        out[tuple(coords)] = 1
        # Pepper mode
        num_pepper = np.ceil(amount * 4096 * (1. - s_vs_p))
        coords = [np.random.randint(0, i ,3) for i in image.shape[1:]]
        out[tuple(coords)] = 0
        out = torch.Tensor(out)
        return out

    # Speckle Noisy
    elif noise_typ =="speckle":
        noisy = np.random.normal(loc=0,scale=1.,size=image.shape)
        noisy = image+ image * 0.1 *noisy
        noisy = torch.Tensor(noisy)
        return noisy

In [8]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
# A small denoising autoencoder model with 0.25 of input data dropped
class DenoisingAE(nn.Module):
    def __init__(self):
        super(DenoisingAE, self).__init__()
        self.add_noise = add_noise
        self.flatten = nn.Flatten()
        self.encode = nn.Sequential(
            nn.Linear(64*64, 250),
            nn.ReLU(),
            nn.Linear(250, 50),
            nn.ReLU(),
            nn.Linear(50, 3),
        )
        self.decode = nn.Sequential(
            nn.Linear(3, 50),
            nn.ReLU(),
            nn.Linear(50, 250),
            nn.ReLU(),
            nn.Linear(250, 64*64)
        )
        
    def forward(self, x):
        x = self.add_noise("gauss",x.cpu()).to(device)
        x = self.flatten(x)
        x = self.encode(x)
        x = self.decode(x)
        x = x.view(-1,1,64,64)
        return x

In [9]:
from torchsummary import summary
model = DenoisingAE().to(device)

summary(model, (1,64,64), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [1, 4096]               0
            Linear-2                   [1, 250]       1,024,250
              ReLU-3                   [1, 250]               0
            Linear-4                    [1, 50]          12,550
              ReLU-5                    [1, 50]               0
            Linear-6                     [1, 3]             153
            Linear-7                    [1, 50]             200
              ReLU-8                    [1, 50]               0
            Linear-9                   [1, 250]          12,750
             ReLU-10                   [1, 250]               0
           Linear-11                  [1, 4096]       1,028,096
Total params: 2,077,999
Trainable params: 2,077,999
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forw

In [18]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001, weight_decay=1e-5)

In [19]:
import time
mae_loss = nn.L1Loss()
start_time = time.time()
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0
    train_psnr = 0
    train_mae = 0
    for batch, x in enumerate(dataloader):
        x = x.to(device)

        # Compute prediction error
        model.train()
        pred = model(x)
        loss = loss_fn(pred, x)
        
        train_loss += loss.item()
        train_psnr += PSNR(pred,x)
        train_mae += mae_loss(pred,x)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= num_batches
    train_psnr /= num_batches
    train_mae /= num_batches     
        
    print("\nLOSS: {:.8f}, TIME: {:.1f}s, psnr : {:1f}, mae : {:1f}".format(train_loss, 
      time.time() - start_time,train_psnr,train_mae))

        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
import math
def PSNR(img1, img2, min_value=0, max_value=1):
    """
    psnr 을 계산해준다.
    이미지가 [0., 255] 이면 min_value=0, max_valu=255 로 해주고,
    이미지가 [-1,1]의 범위에 있으면 min_value=-1, max_valu=1 로 설정 해준다.
    """
    if type(img1) == torch.Tensor:
        mse = torch.mean((img1 - img2) ** 2)
    else:
        mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = max_value - min_value
    return 10 * math.log10((PIXEL_MAX ** 2) / mse)

In [21]:
# Evaluate model accuracy on one batch
start_time = time.time()
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    test_psnr = 0
    test_mae = 0
    with torch.no_grad():
        for x in dataloader:
            x = x.to(device)
            pred = model(x)
            test_loss += loss_fn(pred,x).item()
            test_psnr += PSNR(pred,x)
            test_mae += mae_loss(pred,x)
    test_loss /= num_batches
    test_psnr /= num_batches
    test_mae /= num_batches
    print("\nLOSS: {:.8f}, TIME: {:.1f}s, psnr : {:1f}, mae : {:1f}".format(test_loss, 
      time.time() - start_time,test_psnr,test_mae))

In [22]:
# Train model and output losses for 50 epochs
epochs = 500
for t in range(epochs):
    print(f"Epoch {t+1}")
    train(train_loader, model, loss_fn, optimizer)
    #test(test_loader, model, loss_fn)
print("Done!")

Epoch 1

LOSS: 0.01497662, TIME: 45.4s, psnr : 18.710699, mae : 0.089931
Epoch 2

LOSS: 0.01463105, TIME: 90.6s, psnr : 18.814458, mae : 0.088637
Epoch 3

LOSS: 0.01442424, TIME: 135.9s, psnr : 18.876256, mae : 0.087923
Epoch 4

LOSS: 0.01432443, TIME: 181.2s, psnr : 18.906168, mae : 0.087572
Epoch 5

LOSS: 0.01429093, TIME: 226.5s, psnr : 18.919675, mae : 0.087416
Epoch 6

LOSS: 0.01426291, TIME: 271.7s, psnr : 18.927735, mae : 0.087330
Epoch 7

LOSS: 0.01423557, TIME: 317.0s, psnr : 18.938104, mae : 0.087211
Epoch 8

LOSS: 0.01423210, TIME: 362.3s, psnr : 18.939266, mae : 0.087203
Epoch 9

LOSS: 0.01420266, TIME: 407.6s, psnr : 18.948639, mae : 0.087090
Epoch 10

LOSS: 0.01418025, TIME: 452.9s, psnr : 18.954197, mae : 0.087050
Epoch 11

LOSS: 0.01417009, TIME: 497.6s, psnr : 18.959353, mae : 0.086982
Epoch 12

LOSS: 0.01415855, TIME: 542.4s, psnr : 18.962905, mae : 0.086959
Epoch 13

LOSS: 0.01414949, TIME: 587.6s, psnr : 18.968078, mae : 0.086912
Epoch 14

LOSS: 0.01413725, TIME: 63


LOSS: 0.01356163, TIME: 4928.2s, psnr : 19.168892, mae : 0.084752
Epoch 110

LOSS: 0.01356294, TIME: 4973.4s, psnr : 19.169109, mae : 0.084730
Epoch 111

LOSS: 0.01355167, TIME: 5018.7s, psnr : 19.171335, mae : 0.084712
Epoch 112

LOSS: 0.01354830, TIME: 5063.9s, psnr : 19.173581, mae : 0.084696
Epoch 113

LOSS: 0.01353520, TIME: 5109.2s, psnr : 19.177998, mae : 0.084634
Epoch 114

LOSS: 0.01352098, TIME: 5154.5s, psnr : 19.180957, mae : 0.084612
Epoch 115

LOSS: 0.01354325, TIME: 5199.8s, psnr : 19.175575, mae : 0.084693
Epoch 116

LOSS: 0.01353409, TIME: 5245.0s, psnr : 19.178893, mae : 0.084643
Epoch 117

LOSS: 0.01352737, TIME: 5290.2s, psnr : 19.180389, mae : 0.084622
Epoch 118

LOSS: 0.01352914, TIME: 5335.5s, psnr : 19.180261, mae : 0.084618
Epoch 119

LOSS: 0.01353803, TIME: 5380.7s, psnr : 19.178235, mae : 0.084667
Epoch 120

LOSS: 0.01353760, TIME: 5426.0s, psnr : 19.176514, mae : 0.084650
Epoch 121

LOSS: 0.01352128, TIME: 5471.3s, psnr : 19.182289, mae : 0.084570
Epoch 122


LOSS: 0.01330979, TIME: 9765.7s, psnr : 19.253590, mae : 0.083795
Epoch 217

LOSS: 0.01330706, TIME: 9810.9s, psnr : 19.257387, mae : 0.083757
Epoch 218

LOSS: 0.01332385, TIME: 9856.2s, psnr : 19.249494, mae : 0.083847
Epoch 219

LOSS: 0.01334162, TIME: 9901.4s, psnr : 19.247127, mae : 0.083869
Epoch 220

LOSS: 0.01330975, TIME: 9946.6s, psnr : 19.255549, mae : 0.083790
Epoch 221

LOSS: 0.01330039, TIME: 9991.8s, psnr : 19.256837, mae : 0.083775
Epoch 222

LOSS: 0.01330677, TIME: 10037.0s, psnr : 19.255413, mae : 0.083778
Epoch 223

LOSS: 0.01331215, TIME: 10082.3s, psnr : 19.254010, mae : 0.083816
Epoch 224

LOSS: 0.01330018, TIME: 10127.5s, psnr : 19.257882, mae : 0.083750
Epoch 225

LOSS: 0.01329556, TIME: 10172.7s, psnr : 19.259850, mae : 0.083733
Epoch 226

LOSS: 0.01329558, TIME: 10217.9s, psnr : 19.260375, mae : 0.083717
Epoch 227

LOSS: 0.01328950, TIME: 10263.1s, psnr : 19.260667, mae : 0.083735
Epoch 228

LOSS: 0.01327760, TIME: 10308.3s, psnr : 19.262723, mae : 0.083680
Ep

KeyboardInterrupt: 

In [23]:
torch.save(model,"ae_200.pt")